# **💁🏻🗨️💁🏻‍♂️대화 요약 SOLAR API code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 자료에서는 Solar Chat API를 이용하여 대화 요약 대회를 풀어봅니다.     

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [2]:
!pip install openai

In [1]:
import pandas as pd
import os
import time
from tqdm import tqdm
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.
from openai import OpenAI # openai==1.2.0

### 2) Solar Chat API Client 생성하기
- 앞으로 Solar Chat API를 사용하기 위해 Client를 생성합니다.

In [2]:
UPSTAGE_API_KEY = "up_Uc9JCvebrzPBEu3FN4EJLIZqItftn" # upstage.ai에서 발급받은 API KEY를 입력해주세요.

client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1/solar"
)

### 3) Solar Chat API 사용해보기 (선택)
- 예시 코드를 통해 Solar Chat API를 사용해보세요.

In [3]:
stream = client.chat.completions.create(
    model="solar-1-mini-chat",
    messages=[
      {
        "role": "system",
        "content": "You are a helpful assistant."
      },
      {
        "role": "user",
        "content": "Hello!"
      }
    ],
    stream=True,
)
 
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
 
# Use with stream=False
# print(stream.choices[0].message.content)

Hello! How can I assist you today?

### 4) 데이터 불러오기
- 실험에서 쓰일 데이터를 load합니다.

In [4]:
# 데이터 경로를 지정해줍니다.
DATA_PATH = "../data/"
RESULT_PATH = "./prediction/"

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(DATA_PATH,'train.csv'))
train_df.tail()

FileNotFoundError: [Errno 2] No such file or directory: '../data/train.csv'

In [7]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(DATA_PATH,'dev.csv'))
val_df.tail()

,fname,dialogue,summary,topic
494,dev_495,#Person1#: 이제 새해가 되어서 새로운 시작을 하려고 결심했어. \r\n#P...,#Person1#은 새해에 금연을 하고 커밍아웃하기로 결정했습니다. #Person2...,새해
495,dev_496,"#Person1#: 너, 조랑 결혼했지? \r\n#Person2#: 조? 무슨 말인...",#Person1#은 #Person2#가 조와 결혼했다고 생각했다. #Person2#...,사랑에 빠지다
496,dev_497,"#Person1#: 무엇을 도와드릴까요, 부인?\r\n#Person2#: 몇 주 동...",#Person2#의 차에서 이상한 소리가 납니다. #Person1#는 브레이크를 교...,소음
497,dev_498,"#Person1#: 안녕하세요, 아마존 고객 서비스입니다. 무엇을 도와드릴까요?\n...",#Person2#님이 아마존 고객 서비스에 전화하여 아마존에서 받은 책에 한 페이지...,빠진 페이지
498,dev_499,#Person1#: 여름이 다 되어간다는 게 믿기지 않아.\r\n#Person2#:...,#Person2#는 #Person1#에게 여름 휴가 동안 파티를 도와주는 회사에서 ...,여름 휴가


## 1. Solar Chat API 요약 성능 확인하기
- Solar Chat API을 이용하여 train 및 validation dataset에 포함된 dialogue 샘플을 요약해 봅니다.

In [8]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
rouge = Rouge()
def compute_metrics(pred, gold):
    results = rouge.get_scores(pred, gold, avg=True)
    result = {key: value["f"] for key, value in results.items()}
    return result

In [9]:
# Dialogue를 입력으로 받아, Solar Chat API에 보낼 Prompt를 생성하는 함수를 정의합니다.
def build_prompt(dialogue):
    system_prompt = "You are an expert in the field of dialogue summarization. Please summarize the following dialogue."

    user_prompt = f"Dialogue:\n{dialogue}\n\nSummary:\n"
    
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]

In [10]:
# # Solar Chat API를 활용해 Summarization을 수행하는 함수를 정의합니다.
# def summarization(dialogue):
#     summary = client.chat.completions.create(
#         model="solar-1-mini-chat",
#         messages=build_prompt(dialogue),
#     )

#     return summary.choices[0].message.content

### (선택) parameter 변경하기
- Solar Chat API를 사용할 때, parameter를 변경하여, 다양한 결과를 얻을 수 있습니다.
- Parameter에 대한 자세한 설명은 [여기](https://developers.upstage.ai/docs/apis/chat#request-body)를 참고해주세요.

In [64]:
def summarization(dialogue):
    summary = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=build_prompt(dialogue),
        temperature=0.1,
        top_p=0.1, #0.3
    )

    return summary.choices[0].message.content

Train Dataset을 이용하여 요약이 잘 되는지 확인해 봅니다.

In [41]:
# Train data 중 처음 3개의 대화를 요약합니다.
def test_on_train_data(num_samples=3):
    for idx, row in train_df[:num_samples].iterrows():
        dialogue = row['dialogue']
        summary = summarization(dialogue)
        print(f"Dialogue:\n{dialogue}\n")
        print(f"Pred Summary: {summary}\n")
        print(f"Gold Summary: {row['summary']}\n")
        print("=="*50)

In [42]:
if __name__ == "__main__":
    test_on_train_data()

Dialogue:
#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.

Pred Summary: #Person2#는 건강검진을 받으러 왔으며, 의사는 심각한 질병을 조기에 발견하기 위해 매년 건강검진을 받는 것이 중요하다고 강조합니다. 의사는 #Person2#의 흡연 습관을 발견하고, 이를 끊는 것이 건강에 중요하다고 조언합니다.

Gold Summary: 스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.

Dialogue:
#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?
#Person2#: 안녕하세요, 피터스 박사님. 잘 지냈습니다, 감사합니다. 리키와 함께 백신 접종을 위해 왔습니다.
#Person1#: 좋습니다. 백신 접종 기록을 보니, 

Validation Dataset을 이용하여 요약을 진행하고, 성능을 평가해 봅니다.

In [43]:
# Validation data의 대화를 요약하고, 점수를 측정합니다.
def validate(num_samples=-1):
    val_samples = val_df[:num_samples] if num_samples > 0 else val_df
    
    scores = []
    raw_scores = []
    for idx, row in tqdm(val_samples.iterrows(), total=len(val_samples)):
        dialogue = row['dialogue']
        summary = summarization(dialogue)
        results = compute_metrics(summary, row['summary'])
        avg_score = sum(results.values()) / len(results)
        
        scores.append(avg_score)
        raw_scores.append(results)
    val_avg_score = sum(scores) / len(scores)

    print(f"Validation Average Score: {val_avg_score}")
    return raw_scores

In [15]:
if __name__ == "__main__":
    validate(100) # 100개의 validation sample에 대한 요약을 수행합니다.
    
    # 전체 validation data에 대한 요약을 수행하고 싶은 경우 아래와 같이 실행합니다.
    # validate() 

100%|██████████| 100/100 [02:01<00:00,  1.22s/it]

Validation Average Score: 0.1349404473886698


## 2. Solar Chat API로 요약하기
- Solar Chat API을 이용하여 test dataset에 포함된 dialogue를 요약하고 제출용 파일을 생성합니다.

In [16]:
def inference():
    test_df = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

    summary = []
    start_time = time.time()
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
        dialogue = row['dialogue']
        summary.append(summarization(dialogue))
        
        # Rate limit 방지를 위해 1분 동안 최대 100개의 요청을 보내도록 합니다.
        if (idx + 1) % 100 == 0:
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            if elapsed_time < 60:
                wait_time = 60 - elapsed_time + 5
                print(f"Elapsed time: {elapsed_time:.2f} sec")
                print(f"Waiting for {wait_time} sec")
                time.sleep(wait_time)
            
            start_time = time.time()
    
    output = pd.DataFrame(
        {
            "fname": test_df['fname'],
            "summary" : summary,
        }
    )
    
    if not os.path.exists(RESULT_PATH):
        os.makedirs(RESULT_PATH)
    output.to_csv(os.path.join(RESULT_PATH, "output_solar.csv"), index=False)

    return output

In [17]:
if __name__ == "__main__":
    output = inference()

100%|██████████| 499/499 [10:07<00:00,  1.22s/it]


In [18]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,"이 대화에서, 실장 #Person1#은 직원 #Person2#에게 새로운 사무실 정..."
1,test_1,#Person1#은 #Person2#가 교통 체증으로 인해 늦게 도착한 것에 대해 ...
2,test_2,마샤와 히어로는 2개월 동안 별거한 후 이혼을 신청했습니다. 마샤가 양육권을 가지게...
3,test_3,"이 대화에서, Person1은 Person2의 생일을 축하하고 파티에 초대합니다. ..."
4,test_4,두 사람이 올림픽 공원의 중심인 올림픽 스타디움에 대해 이야기하고 있습니다. 스타디...
...,...,...
494,test_495,잭은 찰리에게 학교 끝나고 자신의 집에서 비디오 게임을 하자고 초대합니다. 그들은 ...
495,test_496,대화에서 Person2는 아내와 함께 레코드 플레이어를 구입한 후 컨트리 음악에 관...
496,test_497,"한 사람이 세탁기와 건조기를 사용하는 방법을 물어보고, 비누를 사용하는 방법에 대해..."
497,test_498,스티브와 매튜는 오랜만에 만나서 이야기를 나눕니다. 스티브는 매튜가 새로운 집을 찾...


## 3. Prompt Engineering
- Prompt engineering을 통해 요약 성능 향상을 시도합니다.

In [71]:
random_seed = 2001
# Few-shot prompt를 생성하기 위해, train data의 일부를 사용합니다.
few_shot_samples = train_df.sample(1, random_state=random_seed)

sample_dialogue1 = few_shot_samples.iloc[0]['dialogue']
sample_summary1 = few_shot_samples.iloc[0]['summary']

print(f"Sample Dialogue1:\n{sample_dialogue1}\n")
print(f"Sample Summary1: {sample_summary1}\n")

Sample Dialogue1:
#Person1#: 디저트로 뭐 드시고 싶으세요?
#Person2#: 뭐 있나요?
#Person1#: 사과 파이, 아이스크림, 초콜릿 케이크, 그리고 과일 칵테일이 있어요.
#Person2#: 사과 파이에 아이스크림을 더해도 될까요?
#Person1#: 물론이죠. 사과 파이는 오늘 아침에 만들었어요. 그래서 아주 신선해요.
#Person2#: 집에서 만든 사과 파이에요. 정말 맛있어요. 레모네이드 한 잔 더 먹어도 될까요?
#Person1#: 그럼요. 직접 가져가세요, 냉장고에 있어요.
#Person2#: 네. 음료수 드실래요?
#Person1#: 네, 아이스티 하나 주세요. 레모네이드 옆에 있어요.
#Person2#: 디저트는 드시나요?
#Person1#: 저도 사과 파이를 먹을 거예요, 하지만 아이스크림은 빼고요. 체중을 조절해야 해서요.

Sample Summary1: #Person2#는 사과 파이에 아이스크림을 더하고 레모네이드 한 잔을 더 원합니다. #Person1#은 아이스티와 아이스크림 없는 사과 파이를 먹을 예정인데, 이는 #Person1#이 체중을 조절해야 하기 때문입니다.



In [72]:
train_df.head()

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스


In [73]:
# Validation data의 대화를 요약하고, 점수를 측정합니다.
def calc_train_score(num_samples=-1):
    train_samples = train_df[:num_samples] if num_samples > 0 else train_df
    
    scores = []
    raw_scores = []
    for idx, row in tqdm(train_samples.iterrows(), total=len(train_samples)):
        dialogue = row['dialogue']
        summary = summarization(dialogue)
        results = compute_metrics(summary, row['summary'])
        avg_score = sum(results.values()) / len(results)
        
        scores.append(avg_score)
        raw_scores.append(results)
        for key, value in results.items():
            train_samples[f'{key}_score'] = value
        
    train_avg_score = sum(scores) / len(scores)

    print(f"Train Average Score: {train_avg_score}")
    return raw_scores

In [22]:
# Prompt를 생성하는 함수를 수정합니다.
def build_prompt(dialogue):
    system_prompt = "You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue."

    user_prompt = (
        "Following the instructions below, summarize the given document.\n"
        "Instructions:\n"
        "1. Read the provided sample dialogue and corresponding summary.\n"
        "2. Read the dialogue carefully.\n"
        "3. Following the sample's style of summary, provide a concise summary of the given dialogue.\n\n"
        "Sample Dialogue:\n"
        f"{sample_dialogue1}\n\n"
        "Sample Summary:\n"
        f"{sample_summary1}\n\n"
        "Dialogue:\n"
        f"{dialogue}\n\n"
        "Summary:\n"
    )
    
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]

In [23]:
# 변경된 prompt를 사용하여, train data 중 처음 3개의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    test_on_train_data()

Dialogue:
#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.

Pred Summary: #Person2#는 #Person1#의 권고에 따라 건강검진을 받습니다. #Person1#은 #Person2#에게 매년 건강검진을 받는 것이 중요하다고 강조하며, 심각한 질병을 조기에 발견하여 예방할 수 있다고 말합니다. 또한, #Person1#은 #Person2#의 흡연 습관에 대해 언급하며, 이를 끊는 것이 건강에 중요하다고 조언합니다.

Gold Summary: 스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.

Dialogue:
#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?
#Person2#: 안녕하세요, 피터스 박사님. 잘 지냈습니다, 감

In [24]:
# 변경된 prompt를 사용하여, validation data의 대화를 요약하고, 점수를 측정합니다.
if __name__ == "__main__":
    scores =validate(100)

100%|██████████| 100/100 [01:25<00:00,  1.17it/s]

Validation Average Score: 0.16816504779852098


다른 방식으로 Few-shot sample을 제공하여 Prompt를 구성해 봅니다.

In [65]:
# Few-shot sample을 다른 방식으로 사용하여 prompt를 생성합니다.
def build_prompt(dialogue):
    system_prompt = "You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue."

    few_shot_user_prompt_1 = (
        "Following the instructions below, summarize the given document.\n"
        "Instructions:\n"
        "1. Read the provided sample dialogue and corresponding summary.\n"
        "2. Read the dialogue carefully.\n"
        "3. Following the sample's style of summary, provide a concise summary of the given dialogue. Be sure that the summary is simple but captures the essence of the dialogue.\n\n"
        "Dialogue:\n"
        f"{sample_dialogue1}\n\n"
        "Summary:\n"
    )
    few_shot_assistant_prompt_1 = sample_summary1
    
    user_prompt = (
        "Dialogue:\n"
        f"{dialogue}\n\n"
        "Summary:\n"
    )
    
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": few_shot_user_prompt_1},
        {"role": "assistant", "content": few_shot_assistant_prompt_1},
        {"role": "user", "content": user_prompt},
    ]

In [66]:
# 변경된 prompt를 사용하여, train data 중 처음 3개의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    test_on_train_data()

Dialogue:
#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.

Pred Summary: 호킨스 의사는 스미스씨에게 매년 건강검진을 받는 것이 중요하다고 강조하며, 심각한 질병을 조기에 발견하여 예방할 수 있다고 말합니다. 의사는 스미스씨의 흡연 습관에 대해 언급하며, 이를 끊는 것이 건강에 중요하다고 조언합니다.

Gold Summary: 스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.

Dialogue:
#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?
#Person2#: 안녕하세요, 피터스 박사님. 잘 지냈습니다, 감사합니다. 리키와 함께 백신 접종을 위해 왔습니다.
#Person1#: 좋습니다. 백신 접종 기록을 보니, 리키는 이

In [62]:
# 변경된 prompt를 사용하여, validation data의 대화를 요약하고, 점수를 측정합니다.
if __name__ == "__main__":
    scores =validate(100)

100%|██████████| 100/100 [01:40<00:00,  1.00s/it]

Validation Average Score: 0.17600246597201838


In [37]:
scores

{'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}

### (선택) 변경된 Prompt로 test dataset에 대한 요약을 진행합니다.
- 변경된 prompt를 통해 점수가 개선되었다면, test dataset에 대한 요약을 진행하고 제출합니다.

In [35]:
# 변경된 prompt를 사용하여, test data의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    output = inference()

  0%|          | 0/499 [00:00<?, ?it/s]

100%|██████████| 499/499 [07:00<00:00,  1.19it/s]


In [63]:
output

,fname,summary
0,test_0,"이 메모는 모든 사무실 통신을 이메일과 공식 메모로 제한하고, 근무 시간 동안 즉시..."
1,test_1,#Person2#는 교통 체증과 오염 문제로 인해 대중교통을 이용하거나 자전거를 타...
2,test_2,마샤와 히어로는 2개월 동안 별거한 후 이혼을 신청했습니다. 마샤가 양육권을 가지게...
3,test_3,"#Person1#은 브라이언의 생일을 축하하고, 둘은 춤을 추며 파티를 즐깁니다."
4,test_4,#Person1#과 #Person2#는 완공 예정인 올림픽 스타디움에 있습니다. 이...
...,...,...
494,test_495,잭은 찰리에게 6시에 아빠를 데리러 가기 전에 2시간 동안 새로운 캐릭터 생성 게임...
495,test_496,"#Person2#는 아내와 함께 컨트리 음악을 듣기 시작하고, 라디오 방송국에서 일..."
496,test_497,#Person1#은 세탁을 해본 적이 없어서 #Person2#에게 도움을 요청합니다...
497,test_498,"#Person1#은 이사를 계획하고 있으며, 분류된 광고를 통해 새로운 거주지를 찾..."
